# 阅读理解模型

## Multi-Hop机制

### 优点

- 多次提取充足的信息

- 语义抽象化

### 方式

- 在句子级别Attention的layer推进

- 在TimeStep状态推进，与主序列维度一致


## Pointer network

### 基本概念

- 从输入序列中找到相应的tokens来作为输出，其利用Attention作为pointer,从输入序列中选择一个位置，并将这个位置所指向的词作为输出

- 个人理解：Pointer Network是将encoder层得到的注意力机制，作用于softmax函数，得到$alpha$概率分布，decoder层直接将其中概率最大的作为当前结果【之前的Seq2Seq的网络还会将$alpha$对各个输入元素做加权和】

### 基本原理

- 传统的注意力机制

![title](img/pn1.png)

- Pointer network的注意力机制

![title](img/pn2.png)


### 优点

- 提供了一种新视角去理解 Attention，把 Attention 作为一种求分布的手段。

- 对于输出字典长度不固定问题提供了一种新的解决方案。

- 将输入作为输出的一种补充手段，让输出部分可以更好的引入输入部分的信息。

### 论文地址

- https://arxiv.org/abs/1506.03134

- https://arxiv.org/abs/1608.07905

### 相关博文

- https://zhuanlan.zhihu.com/p/48959800

- https://lianhaimiao.github.io/2018/08/21/Pointer-Network-%E7%9A%84%E4%B8%80%E4%BA%9B%E7%90%86%E8%A7%A3/


## R-Net模型

### 架构图

![title](img/rnet1.png)

### 架构简介

- R-Net一共分为四层，由下至上，分别是编码层、交互层（其中交互层又分为文章问题交互层以及文章的自交互层）、输出层。

- 其中，编码层是一个Encoding过程，是双向RNN（这里采用GRU）,主要负责将输入的文章【Passage】和问题【Query】进行词向量转换

- 第二层是文章和问题的交互层，计算Q\P注意力机制

- 第三层是文章的自交互层，计算QP\P注意力机制

- 第四层是输出层，这里加入的Poniter Newtwork，来预测P(begin) & P(end)作为答案起始位置


### 基础原理

- 编码层 Question-Passage Encoding

> 最终每个词有一个词级别的向量，一个char级别的向量。

> 其中char级别的向量用来处理，例如 OOV、错误纠正等问题

> 之后通过一个双向RNN网络，得到文章和问题的向量表示

> - 这里的BiRNN使用GRU,代替了传统Attention中的LSTM,因为GRU少一个门，计算速度快

![title](img/rnet2.png)

- 交互层 Question-Passage Matching

> 将问题向量结果融合到文章中，从而得到问题向量下文章的语义理解

> 为了更好的提取和文章有关的问题信息，加入了门控机制$g_{t}$，主要是通过sigmoid函数输出概率分布，将相关度小的计算出相对小的概率

> 采用的是RNN网络

![title](img/rnet3.png)

![title](img/rnet4.png)

- 交互层  self-Matching

> 将上一层的输出结果做一个self-to-self的matching

> 此时所有的问题内容已经到融合到了文章中

> 这一层没有门控机制

> 采用BiRNN网络

![title](img/rnet5.png)

- 输出层 Answer Prediction

> 借鉴Pointer Network思想.最终输出的是问题的开始和结束位置

> 在这里融合的问题向量的加权重要性向量表示

![title](img/rnet6.png)

![title](img/rnet7.png)


### 特点

- 改进了RNN在机器阅读理解中的应用

- 将注意力机制融入RNN

- 门控循环神经网络

### 论文地址

- https://www.microsoft.com/en-us/research/wp-content/uploads/2017/05/r-net.pdf


### 相关博文

- https://zhuanlan.zhihu.com/p/36855204


## FusionNet

- 这个网络的提出贡献的两个重要的思想

> 单词历史

> 全关注注意力

### 基本概念


### 网络架构


### 基本原理


### 论文地址

- https://arxiv.org/pdf/1711.07341.pdf


### 相关博文



## QA-Net

